In [2]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

import os 
import sys 
import json

home = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..'))
pths = [os.path.join(home, 'astrobot')]

for pth in pths: 
    if pth not in sys.path: sys.path.insert(0, pth)

from astrobot.util import *
from astrobot.geomdataframe import GeomDataFrame 
from astrobot.r import R
import astrobot.viz4 as viz
from astrobot import bem_util, geom_util, mtx_util

import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
import itertools

# Results
from ladybug.sql import SQLiteResult
from honeybee_energy.result import match 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
deeprad_model_dir = os.path.abspath(os.path.join(os.getcwd(), '../../..', 'deeprad/data/models/'))
floorplan_id = 'floorplan_1754'

gen_dir = os.path.join(deeprad_model_dir, floorplan_id)
sim_dir = os.path.join(deeprad_model_dir, floorplan_id, 'simulation')

print('gen_dir: ', gen_dir)
print('sim_dir: ', sim_dir)

RDD_SRF_DICT = {
    'srf_out_temp': 'Surface Outside Face Temperature',
    'srf_in_temp': 'Surface Inside Face Temperature',
    'srf_heat_loss': 'Surface Window Heat Loss Energy',
    'srf_heat_gain': 'Surface Window Heat Gain Energy',
    'srf_win_sol': 'Surface Window Transmitted Solar Radiation Energy'
}

gen_dir:  /mnt/c/Users/Administrator/master/git/deeprad/data/models/floorplan_1754
sim_dir:  /mnt/c/Users/Administrator/master/git/deeprad/data/models/floorplan_1754/simulation


In [54]:
r = R(pd.read_pickle(os.path.join(sim_dir, 'rmod_gen.pkl')))
r.spc = pd.read_pickle(os.path.join(sim_dir, 'rspc_gen.pkl'))
r.srf = pd.read_pickle(os.path.join(sim_dir, 'rsrf_gen.pkl'))
r.win = pd.read_pickle(os.path.join(sim_dir, 'rwin_gen.pkl'))


In [51]:
#rdd_srf_var = RDD_SRF_DICT['srf_win_sol']
rdd_srf_var = RDD_SRF_DICT['srf_out_temp']
rdatas = []
print(rdd_srf_var)
for i, mod in enumerate(r.mod['mod']):
    sql_fpath = os.path.join(sim_dir, '{}_model'.format(i), 'OpenStudio', '0_building', 'run', 'eplusout.sql')
    sql = SQLiteResult(sql_fpath)
    rdata = sql.data_collections_by_output_name(rdd_srf_var)
    rdatas.extend(rdata)    

Surface Outside Face Temperature


In [58]:
#r.srf['srf_out_temp'] = [rd.total for f, rd in match.match_faces_to_data(rdatas, r.srf.srf_geom)]
matched = match.match_faces_to_data(rdatas, r.srf.srf_geom)

#r.win['sol'] = [np.round(r.total / 72.0, 2) for f, r in matched]
#r.win[r.win.columns]
r.srf['out_temp'] = [np.round(r.average, 2) for _, r in matched]
r.srf.head()


,type,bc,out_temp
idx,,,
0,Floor,Adiabatic,20.96
1,Wall,Outdoors,17.03
2,Wall,Outdoors,14.12
3,Wall,Outdoors,17.12
4,RoofCeiling,Adiabatic,21.08
